In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from numpy.random import randn
from numpy.matlib import repmat

from scipy.stats import norm
from scipy.optimize import fmin
from scipy.special import erf

from patsy import dmatrices

from sklearn import metrics
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer

In [2]:
a = pd.DataFrame([[np.nan, 2, 3], [4, np.nan, 6]])

a[0] = a[0].fillna(0)

In [3]:
def array_vector(col):
    return np.array(str(col))

arrayerize = np.vectorize(array_vector)

def one_hot_encode(df, column, labels_column=None, whitelist=[]):
    # This is gross but since strings are iterable, we have to wrap them in a list
    # in order for the binarizer to parse the labels as strings and not chars
    labels = arrayerize(pd.DataFrame(df[column]))
    terms = arrayerize(pd.DataFrame(list(set(df[column]))))

    mlb = MultiLabelBinarizer()
    mlb.fit(terms)
    mlb.transform(labels)
    columns = [ f'{column}-{classname}' for classname in  mlb.classes_]

    encoded = pd.DataFrame(mlb.transform(labels), columns=columns, index=df[column].index)

    df.drop(column, axis=1, inplace=True)
    return df.join(encoded)

In [4]:
firms = pd.read_csv('../match-data/match-v1/firms.csv')
jobs = pd.read_csv('../match-data/match-v1/job-openings.csv')
jobs_parent = pd.read_csv('../match-data/match-v1/job-openings-parent.csv')
job_seekers = pd.read_csv('../match-data/match-v1/job-seekers.csv')
matches = pd.read_csv('../match-data/match-v1/matches.csv')
matches_parent = pd.read_csv('../match-data/match-v1/matches-parent.csv')


matches_merged = pd.merge(matches, matches_parent, on='number')
jobs_merged = pd.merge(jobs, jobs_parent, on='number')

In [5]:
categorical_columns = ["gender", "highest_edu_level", "nationality", "gendermix_not_allowed", "benefit1", "benefit2", "city", "english_proficiency", "impairments", "major", "opposite_gender_coworkers", "opposite_gender_manager", "first_job_field_preference", "second_job_field_preference", "bus_covered", "childcare_subsidy_offered", "dorm_covered", "driving_ability_required", "education_required", "english_proficiency_required", "female_requied", "free_meals_at_wok", "health_insurance_offered", "hearing_disability_accepted", "housing_subsidy_offered", "it_proficiency_required", "job_category", "job_description", "job_production", "jordanian_experience_required", "literacy_required", "male_required", "meal_subsidy_offered", "night_shifts_required", "noncognitive_skill_preference1", "noncognitive_skill_preference2", "numeracy_requied", "physical_disability_accepted", "physical_work_abilities_required", "problem_solving_required", "school_subsidy_offered", "specialization_required", "speech_disability_accepted", "syrian_considered", "transport_subsidy_offered", "visual_disability_accepted", "work_permit_offered",]
all_columns = ["age", "gender", "highest_edu_level", "will_work_night_shift", "nationality", "gendermix_not_allowed", "will_work_qiz", "arab_coworkers", "benefit1", "benefit2", "city", "daily_hours_willing_to_work", "days_willing_train_unpaid", "distance_willing_to_travel", "english_proficiency", "experience_clerical_work", "experience_factory", "experience_management_work", "experience_manual_labor", "experience_professional_work", "follow_up_agreement", "has_job", "impairments", "major", "nonarab_coworkers", "opposite_gender_coworkers", "opposite_gender_manager", "weekly_days_willing_to_work", "will_live_in_dorm", "will_train_unpaid", "years_education", "years_exp", "first_job_field_preference", "rwage1", "second_job_field_preference", "hh_income", "hired_yes_no", "interest_applying", "num_children", "personal_income", "bus_covered", "childcare_subsidy_offered", "dorm_covered", "driving_ability_required", "education_required", "english_proficiency_required", "female_requied", "free_meals_at_wok", "health_insurance_offered", "hearing_disability_accepted", "housing_subsidy_offered", "it_proficiency_required", "job_category", "job_description", "job_production", "jordanian_experience_required", "literacy_required", "male_required", "meal_subsidy_offered", "night_shifts_required", "noncognitive_skill_preference1", "noncognitive_skill_preference2", "num_vacancies", "numeracy_requied", "physical_disability_accepted", "physical_work_abilities_required", "problem_solving_required", "school_subsidy_offered", "specialization_required", "speech_disability_accepted", "syrian_considered", "transport_subsidy_offered", "visual_disability_accepted", "wage_offered", "work_permit_offered", "years_experience_required",]
scalar_columns = ["age", "daily_hours_willing_to_work", "days_willing_train_unpaid", "distance_willing_to_travel", "years_education", "years_exp", "rwage1", "hh_income", "num_children", "personal_income", "num_vacancies", "wage_offered"]

In [6]:
job_seekers['parent_case_id'] = job_seekers['caseid']

In [7]:
merged = pd.merge(job_seekers, matches_merged, on='parent_case_id')
merged = pd.merge(merged, jobs_merged, on='job_id')

merged.to_csv('../match-data/match-v1/merged.csv')

In [8]:
formatted = pd.DataFrame()
for col in all_columns:
    formatted[col] = merged[col]


for col in categorical_columns:
    formatted = one_hot_encode(formatted, col)

for col in formatted.columns:
    formatted[col] = formatted[col].replace(['---'], 0)
    
formatted.to_csv('../match-data/match-v1/formatted.csv')

In [9]:
y = formatted['hired_yes_no']
y = y.fillna(0)
formatted = formatted.drop(['hired_yes_no'], axis=1)


In [10]:
for col in formatted.columns:
    if col in scalar_columns:
        formatted[col] = formatted[col].astype(float)
        mean = formatted[col].mean()
        formatted[col] = formatted[col].replace(['---'], mean)
        formatted[col] = formatted[col].fillna(mean)
    else:
        formatted[col] = formatted[col].fillna(0)
        formatted[col] = formatted[col].astype(int)
        formatted[col] = formatted[col].replace(['---'], 0)

formatted.to_csv('../match-data/match-v1/X.csv')

In [11]:
y=y.astype('int')
y.to_csv('../match-data/match-v1/y.csv')

In [12]:
model = LogisticRegression(max_iter=1000)
model.fit(formatted, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
coef_dict = {}
for coef, feat in zip(model.coef_[0], formatted.columns):
    coef_dict[feat] = coef

In [14]:
c = {}
for k, v in coef_dict.items():
    c[k] = [v]
    

coef_frame = pd.DataFrame.from_dict(c)
sorted_frame = coef_frame.columns[coef_frame.ix[coef_frame.last_valid_index()].argsort()]
coef_frame.to_csv('../match-data/match-v1/coefs.csv')

/home/nolski/code/tashbeek/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [15]:
sorted_frame
final_sorted = pd.DataFrame()
for key in sorted_frame:
    final_sorted[key] = coef_frame[key]

final_sorted.to_csv('../match-data/match-v1/coefs_sorted.csv')

In [16]:
from sklearn.feature_selection import chi2
scores, pvalues = chi2(formatted, y)

In [17]:
p_dict = {}
for pvalue, feat in zip(pvalues, formatted.columns):
    p_dict[feat] = pvalue
    
c = {}
for k, v in p_dict.items():
    c[k] = [v]
    

coef_frame = pd.DataFrame.from_dict(c)
sorted_frame = coef_frame.columns[coef_frame.ix[coef_frame.last_valid_index()].argsort()]
coef_frame.to_csv('../match-data/match-v1/pvalues.csv')

/home/nolski/code/tashbeek/venv/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


In [18]:
sorted_frame
final_sorted = pd.DataFrame()
for key in sorted_frame:
    final_sorted[key] = coef_frame[key]

final_sorted.to_csv('../match-data/match-v1/pvalues_sorted.csv')